In [69]:
import pandas as pd
import csv
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/sf_airbnb/{}.csv'

listings_summary = file_url.format('listings_summary')
reviews_file = file_url.format('reviews_details')
calendar_file = file_url.format('calendar_details')

In [3]:
df_listings = pd.read_csv(listings_summary, parse_dates=['last_review'])

In [4]:
df_listings.shape

(6633, 16)

In [5]:
df_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,958,"Bright, Modern Garden Unit - 1BR/1B",1169,Holly,NaN,Western Addition,37.769310,-122.433856,Entire home/apt,170,1,152,2018-07-29,1.38,1,122
1,5858,Creative Sanctuary,8904,Philip And Tania,NaN,Bernal Heights,37.745112,-122.421018,Entire home/apt,235,30,112,2017-08-06,0.99,1,365
2,7918,A Friendly Room - UCSF/USF - San Francisco,21994,Aaron,NaN,Haight Ashbury,37.766690,-122.452505,Private room,65,32,17,2016-11-21,0.16,9,365
3,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,21994,Aaron,NaN,Haight Ashbury,37.764872,-122.451828,Private room,65,32,7,2017-08-12,0.15,9,328
4,8339,Historic Alamo Square Victorian,24215,Rosy,NaN,Western Addition,37.775249,-122.436374,Entire home/apt,675,7,26,2017-11-10,0.24,2,78


In [6]:
df_listings.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [7]:
irrlevant_columns_to_drop = ['name', 'host_id', 'host_name', 'neighbourhood_group']

In [8]:
df_listings_smaller = df_listings.drop(irrlevant_columns_to_drop, axis=1)

In [9]:
df_listings_smaller.dtypes

id                                         int64
neighbourhood                             object
latitude                                 float64
longitude                                float64
room_type                                 object
price                                      int64
minimum_nights                             int64
number_of_reviews                          int64
last_review                       datetime64[ns]
reviews_per_month                        float64
calculated_host_listings_count             int64
availability_365                           int64
dtype: object

In [10]:
datetime.now().year

2018

In [11]:
def duration(starting_time):
    duration =0
    starting_year = starting_time.year
    duration = datetime.now().year - starting_year + 1
    return duration

In [12]:
df_listings_smaller['yrs_since_last_review'] = df_listings_smaller.last_review.apply(lambda x: duration(x))

In [13]:
df_listings_smaller.head()

,id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,yrs_since_last_review
0,958,Western Addition,37.769310,-122.433856,Entire home/apt,170,1,152,2018-07-29,1.38,1,122,1.0
1,5858,Bernal Heights,37.745112,-122.421018,Entire home/apt,235,30,112,2017-08-06,0.99,1,365,2.0
2,7918,Haight Ashbury,37.766690,-122.452505,Private room,65,32,17,2016-11-21,0.16,9,365,3.0
3,8142,Haight Ashbury,37.764872,-122.451828,Private room,65,32,7,2017-08-12,0.15,9,328,2.0
4,8339,Western Addition,37.775249,-122.436374,Entire home/apt,675,7,26,2017-11-10,0.24,2,78,2.0


In [14]:
df_listings_smaller.drop('last_review', axis=1, inplace=True)

In [15]:
df_listings_smaller.isnull().sum()

id                                   0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                 1142
calculated_host_listings_count       0
availability_365                     0
yrs_since_last_review             1142
dtype: int64

In [16]:
df_listings_smaller.dropna(inplace=True)

In [17]:
df_listings_smaller.shape

(5491, 12)

In [18]:
df_listings_smaller.dtypes

id                                  int64
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
yrs_since_last_review             float64
dtype: object

In [19]:
categorical_data = ['neighbourhood', 'room_type']

In [20]:
df_listings_cleaned = pd.get_dummies(df_listings_smaller, prefix=categorical_data, columns=categorical_data,)

In [21]:
df_listings_cleaned.columns

Index(['id', 'latitude', 'longitude', 'price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365',
       'yrs_since_last_review', 'neighbourhood_Bayview',
       'neighbourhood_Bernal Heights', 'neighbourhood_Castro/Upper Market',
       'neighbourhood_Chinatown', 'neighbourhood_Crocker Amazon',
       'neighbourhood_Diamond Heights', 'neighbourhood_Downtown/Civic Center',
       'neighbourhood_Excelsior', 'neighbourhood_Financial District',
       'neighbourhood_Glen Park', 'neighbourhood_Golden Gate Park',
       'neighbourhood_Haight Ashbury', 'neighbourhood_Inner Richmond',
       'neighbourhood_Inner Sunset', 'neighbourhood_Lakeshore',
       'neighbourhood_Marina', 'neighbourhood_Mission',
       'neighbourhood_Nob Hill', 'neighbourhood_Noe Valley',
       'neighbourhood_North Beach', 'neighbourhood_Ocean View',
       'neighbourhood_Outer Mission', 'neighbourhood_Outer Richmond',
       'neighbourhood_O

In [22]:
df_listings_cleaned.head()

,id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,yrs_since_last_review,...,neighbourhood_Russian Hill,neighbourhood_Seacliff,neighbourhood_South of Market,neighbourhood_Twin Peaks,neighbourhood_Visitacion Valley,neighbourhood_West of Twin Peaks,neighbourhood_Western Addition,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,958,37.769310,-122.433856,170,1,152,1.38,1,122,1.0,...,0,0,0,0,0,0,1,1,0,0
1,5858,37.745112,-122.421018,235,30,112,0.99,1,365,2.0,...,0,0,0,0,0,0,0,1,0,0
2,7918,37.766690,-122.452505,65,32,17,0.16,9,365,3.0,...,0,0,0,0,0,0,0,0,1,0
3,8142,37.764872,-122.451828,65,32,7,0.15,9,328,2.0,...,0,0,0,0,0,0,0,0,1,0
4,8339,37.775249,-122.436374,675,7,26,0.24,2,78,2.0,...,0,0,0,0,0,0,1,1,0,0


In [23]:
# Create correlation matrix
corr_matrix = df_listings_cleaned.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [24]:
to_drop

['room_type_Private room']

In [26]:
# Drop features 
df_listings_cleaned.drop(to_drop, axis=1, inplace=True)

In [27]:
df_listings_cleaned.columns

Index(['id', 'latitude', 'longitude', 'price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365',
       'yrs_since_last_review', 'neighbourhood_Bayview',
       'neighbourhood_Bernal Heights', 'neighbourhood_Castro/Upper Market',
       'neighbourhood_Chinatown', 'neighbourhood_Crocker Amazon',
       'neighbourhood_Diamond Heights', 'neighbourhood_Downtown/Civic Center',
       'neighbourhood_Excelsior', 'neighbourhood_Financial District',
       'neighbourhood_Glen Park', 'neighbourhood_Golden Gate Park',
       'neighbourhood_Haight Ashbury', 'neighbourhood_Inner Richmond',
       'neighbourhood_Inner Sunset', 'neighbourhood_Lakeshore',
       'neighbourhood_Marina', 'neighbourhood_Mission',
       'neighbourhood_Nob Hill', 'neighbourhood_Noe Valley',
       'neighbourhood_North Beach', 'neighbourhood_Ocean View',
       'neighbourhood_Outer Mission', 'neighbourhood_Outer Richmond',
       'neighbourhood_O

In [71]:
test=['id', 'latitude', 'longitude', 'price']

In [72]:
X = df_listings_cleaned.drop(test, axis=1)
y = df_listings_cleaned['price']

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [74]:
ss = StandardScaler()
X_train_trans = ss.fit_transform(X_train)
X_test_trans = ss.transform(X_test)

In [75]:
lm = linear_model.LinearRegression()
lm.fit(X_train_trans, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [76]:
lm.score(X_test_trans, y_test)

0.15315145685105247

In [77]:
model=sm.OLS(y_train, sm.add_constant(X_train_trans))
fit=model.fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                     8.492
Date:                Thu, 13 Sep 2018   Prob (F-statistic):           8.58e-49
Time:                        10:38:31   Log-Likelihood:                -27363.
No. Observations:                3843   AIC:                         5.481e+04
Df Residuals:                    3800   BIC:                         5.508e+04
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        209.0874      4.854     43.071      0.000     199.570     218.605
x1            -2.2048      4.871     -0.453      0.651     -11.754       7.344
x2           -15.7446      6.349     -2.480      0.013     -28.192      -3.297
x3           -12.9162      6.725     -1.921      0.055     -26.102       0.269
x4           -21.4462      5.316     -4.034      0.000     -31.868     -11.024
x5           -10.3098      5.121     -2.013      0.044     -20.351      -0.269
x6             3.8634      5.149      0.750      0.453      -6.232      13.959
x7           -10.1639      4.787     -2.123      0.034     -19.550      -0.778
x8           -11.9507      4.571     -2.614      0.009     -20.913      -2.988
x9             3.4858      4.573      0.762      0.446      -5.481      12.453
x10           -2.1459      4.871     -0.440      0.660     -11.697       7.405
x11           -5.0335      4.836     -1.041      0.298     -14.516       4.449
x12            1.8918      4.839      0.391      0.696      -7.596      11.380
x13           -5.4802      4.740     -1.156      0.248     -14.773       3.813
x14          -11.9705      4.761     -2.515      0.012     -21.304      -2.637
x15            2.3061      4.844      0.476      0.634      -7.191      11.803
x16           -2.2985      4.815     -0.477      0.633     -11.739       7.142
x17            2.8567      4.851      0.589      0.556      -6.653      12.367
x18           -1.5141      4.653     -0.325      0.745     -10.637       7.609
x19           -3.2981      4.695     -0.702      0.482     -12.504       5.908
x20           -4.5981      4.766     -0.965      0.335     -13.942       4.746
x21           -4.5731      4.874     -0.938      0.348     -14.129       4.983
x22           11.0338      4.746      2.325      0.020       1.730      20.338
x23           -3.1676      4.375     -0.724      0.469     -11.746       5.411
x24           10.9702      4.754      2.307      0.021       1.649      20.291
x25            5.1812      4.646      1.115      0.265      -3.928      14.290
x26           -2.0063      4.763     -0.421      0.674     -11.345       7.332
x27           -7.8580      4.810     -1.634      0.102     -17.289       1.573
x28           -6.0513      4.742     -1.276      0.202     -15.349       3.246
x29           -4.6112      4.778     -0.965      0.335     -13.979       4.757
x30          -10.0347      4.726     -2.123      0.034     -19.300      -0.769
x31            7.2439      4.746      1.526      0.127      -2.061      16.549
x32            7.2609      4.776      1.520      0.129      -2.103      16.625
x33            1.7991      4.712      0.382      0.703      -7.440      11.038
x34         2.461e-15   1.62e-15      1.519      0.129   -7.16e-16    5.64e-15
x35           12.5846      4.838      2.601      0.009       3.099      22.071
x3